In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import seaborn as sns
import timedelta

plt.rcParams['font.family'] = 'Malgun Gothic'  # or 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False




In [2]:
import numpy as np
from scipy.spatial import cKDTree
pd.set_option('display.max_columns', None)


In [3]:
loc_df = pd.read_csv("./weather_observed/meta-loc-edited.csv", encoding='cp949')
loc_gdf = gpd.GeoDataFrame(
    loc_df, geometry=gpd.points_from_xy(loc_df.경도, loc_df.위도), crs="EPSG:4326"
)
loc_gdf.지점 = loc_gdf.지점.apply(str)

In [4]:
loc_gdf

,지점,시작일,종료일,지점명,위도,경도,geometry
0,12,2007-11-30,NaN,안면도(감),36.5333,126.3167,POINT (126.3167 36.5333)
1,96,2020-10-23,NaN,독도,37.2395,131.8698,POINT (131.8698 37.2395)
2,116,2023-10-21,NaN,관악(레),37.4453,126.9640,POINT (126.964 37.4453)
3,144,1993-03-16,2011-12-09,군산(레),36.0123,126.7834,POINT (126.7834 36.0123)
4,160,2020-10-23,NaN,부산(레),35.1188,129.0000,POINT (129 35.1188)
...,...,...,...,...,...,...,...
567,992,2021-11-10,NaN,하빈,35.9062,128.4464,POINT (128.4464 35.9062)
568,993,2024-04-06,NaN,제주금악,33.3423,126.3099,POINT (126.3099 33.3423)
569,994,2022-05-02,NaN,심포,35.8545,126.6421,POINT (126.6421 35.8545)
570,995,2023-12-19,NaN,오천,35.9286,129.3822,POINT (129.3822 35.9286)


In [5]:
# 기상 관측 정보 불러오기 
weather = gpd.read_file("./weather_observed/whetherDataEH.csv", encoding='cp949')
# 기상 관측 정보에 있는 지점들 찾기
loc_gdf_needed = loc_gdf[loc_gdf.지점.isin(weather.지점.unique())]
loc_gdf_needed

,지점,시작일,종료일,지점명,위도,경도,geometry
0,12,2007-11-30,NaN,안면도(감),36.5333,126.3167,POINT (126.3167 36.5333)
1,96,2020-10-23,NaN,독도,37.2395,131.8698,POINT (131.8698 37.2395)
2,116,2023-10-21,NaN,관악(레),37.4453,126.9640,POINT (126.964 37.4453)
4,160,2020-10-23,NaN,부산(레),35.1188,129.0000,POINT (129 35.1188)
5,175,2024-10-15,NaN,진도(레),34.4721,126.3238,POINT (126.3238 34.4721)
...,...,...,...,...,...,...,...
567,992,2021-11-10,NaN,하빈,35.9062,128.4464,POINT (128.4464 35.9062)
568,993,2024-04-06,NaN,제주금악,33.3423,126.3099,POINT (126.3099 33.3423)
569,994,2022-05-02,NaN,심포,35.8545,126.6421,POINT (126.6421 35.8545)
570,995,2023-12-19,NaN,오천,35.9286,129.3822,POINT (129.3822 35.9286)


In [6]:
# 읍면동 경계 불러오기 
emd = gpd.read_file(f"./emd/emd.shp", encoding="euc-kr")
emd.set_crs(epsg=5179, inplace=True)
emd = emd.to_crs(loc_gdf.crs)

In [7]:
# 읍면동 별 가장 가까운 관측 지점 매핑 
nearest_weather_loc = emd.sjoin_nearest(loc_gdf_needed, how = "left")
nearest_weather_loc = nearest_weather_loc[['EMD_CD','지점']]
nearest_weather_loc.EMD_CD = nearest_weather_loc.EMD_CD.apply(int)
nearest_weather_loc.지점 = nearest_weather_loc.지점.apply(int)

c:\Users\sc.lee\miniconda3\envs\wildfire_env\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [8]:
Result = pd.read_csv("./Result_data.csv")
Result.head()

,EMD_CD,DMCLS_CD_wmode,AGCLS_CD_wmode,frtp_avg,dnst_avg,Area,geometry,mtn_area,Mtn_Ratio,human_adj_area,human_mtn_Ratio,human_Total_Ratio,ctr_mean,ctr_std,n_ctrs,Fire_Count
0,11110101,23,4,0.125,0.6,792617.902967,POLYGON ((953700.0221347539 1954605.0654306228...,67529.485826,0.085198,36117.996122,0.534848,0.045568,173.809524,73.940935,21,0
1,11110102,23,4,0.125,0.6,98821.715201,POLYGON ((953233.4654317541 1953996.9839332262...,59869.101208,0.605829,40495.878201,0.676407,0.409787,87.500000,32.274861,4,0
2,11110103,23,4,0.125,0.6,73271.654659,POLYGON ((953560.2281430522 1954257.4664824517...,776.906691,0.010603,0.000000,0.000000,0.000000,50.000000,0.000000,1,0
3,11110109,23,4,0.125,0.6,258561.672009,POLYGON ((952981.2076348594 1953604.3112467206...,79017.474842,0.305604,77079.210797,0.975470,0.298108,160.000000,71.879529,10,0
4,11110111,23,4,0.125,0.6,417438.349374,"POLYGON ((952769.057857475 1954155.9539321065,...",113433.905865,0.271738,113433.905865,1.000000,0.271738,187.500000,84.210131,14,0


In [9]:
weather_stats = pd.read_csv("./location_weather_stats.csv")

In [10]:
weather_stats = weather_stats[(weather_stats.지점).isin(nearest_weather_loc.지점)]

In [11]:
weather_by_emd = nearest_weather_loc.merge(weather_stats, how = "left")

In [12]:
weather_by_emd.drop(columns=["지점명", "지점"], inplace= True)
Result = Result.merge(weather_by_emd, how = "left")

In [13]:
Result.to_csv('Result&Weather_data.csv', index=False)